# Similarity search for motif mining

In this tutorial, we demonstrate how to utilize the similarity functionality to discover images with similar motifs from a collection of Reddit images. We employ the classic `SIFT` feature to identify images with a strikingly similar appearance (image-level pipeline).

Additionally, we extend the pipeline by incorporating an object detection model, `YOLO`, in combination with the SIFT feature. This enables us to identify objects within the images that exhibit a similar appearance (object-level similarity).

To illustrate the seamless integration of different vector stores, we leverage the power of multiple vector stores, namely `FAISS` and `QDRANT`, within evadb. This demonstrates the ease with which you can utilize diverse vector stores to construct indexes, enhancing your similarity search experience.

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/georgia-tech-db/eva/blob/master/tutorials/11-similarity-search-for-motif-mining.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run on Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/georgia-tech-db/eva/blob/master/tutorials/11-similarity-search-for-motif-mining.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/georgia-tech-db/eva/raw/master/tutorials/11-similarity-search-for-motif-mining.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" /> Download notebook</a>
  </td>
</table><br><br>

### Connect to EvaDB


In [1]:
%pip install --quiet "evadb[vision,document,notebook]"
%pip install --quiet kornia qdrant_client
import evadb
cursor = evadb.connect().cursor()

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-tools 1.56.0 requires protobuf<5.0dev,>=4.21.6, but you have protobuf 3.20.1 which is incompatible.
ray 2.4.0 requires grpcio<=1.51.3,>=1.42.0; python_version >= "3.10" and sys_platform != "darwin", but you have grpcio 1.56.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ray 2.4.0 requires grpcio<=1.51.3,>=1.42.0; python_version >= "3.10" and sys_platform != "darwin", but you have grpcio 1.56.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


### Download reddit dataset

In [ ]:

!wget -nc https://www.dropbox.com/scl/fo/fcj6ojmii0gw92zg3jb2s/h\?dl\=1\&rlkey\=j3kj1ox4yn5fhonw06v0pn7r9 -O reddit-images.zip
!unzip -o reddit-images.zip -d reddit-images

--2023-06-24 12:06:12--  https://www.dropbox.com/scl/fo/fcj6ojmii0gw92zg3jb2s/h?dl=1&rlkey=j3kj1ox4yn5fhonw06v0pn7r9
Resolving www.dropbox.com (www.dropbox.com)... 

162.125.9.18, 2620:100:601f:18::a27d:912
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.18|:443... connected.
HTTP request sent, awaiting response... 

302 Found
Location: https://ucb5dd5994d9c04177dc93396946.dl.dropboxusercontent.com/zip_download_get/Bh_JNE-Dsk_vODTmqnhrF1AqyGJHJ1bhZHjFi6Og26OcQcltrTbYIxqAx1YZ5Y5nX9vVRVkPfsGmFhEOwnMd1DoSlPTim70W35c7Dn4OV6UDFw# [following]
--2023-06-24 12:06:13--  https://ucb5dd5994d9c04177dc93396946.dl.dropboxusercontent.com/zip_download_get/Bh_JNE-Dsk_vODTmqnhrF1AqyGJHJ1bhZHjFi6Og26OcQcltrTbYIxqAx1YZ5Y5nX9vVRVkPfsGmFhEOwnMd1DoSlPTim70W35c7Dn4OV6UDFw
Resolving ucb5dd5994d9c04177dc93396946.dl.dropboxusercontent.com (ucb5dd5994d9c04177dc93396946.dl.dropboxusercontent.com)... 

162.125.9.15, 2620:100:601f:15::a27d:90f
Connecting to ucb5dd5994d9c04177dc93396946.dl.dropboxusercontent.com (ucb5dd5994d9c04177dc93396946.dl.dropboxusercontent.com)|162.125.9.15|:443... connected.
HTTP request sent, awaiting response... 

200 OK
Length: 4397651 (4.2M) [application/zip]
Saving to: ‘reddit-images.zip’


reddit-images.zip     0%[                    ]       0  --.-KB/s               


reddit-images.zip    73%[=============>      ]   3.07M  8.20MB/s               
reddit-images.zip   100%[===================>]   4.19M  10.9MB/s    in 0.4s    

2023-06-24 12:06:14 (10.9 MB/s) - ‘reddit-images.zip’ saved [4397651/4397651]



Archive:  reddit-images.zip
mapname:  conversion of  failed
 extracting: reddit-images/g348_d7jgzgf.jpg  
 extracting: reddit-images/g348_d7jphyc.jpg  
 extracting: reddit-images/g348_d7ju7dq.jpg  
 extracting: reddit-images/g348_d7jhhs3.jpg  
 extracting: reddit-images/g1074_d4n1lmn.jpg  
 extracting: reddit-images/g1074_d4mxztt.jpg  
 extracting: reddit-images/g1074_d4n60oy.jpg  
 extracting: reddit-images/g1074_d4n6fgs.jpg  
 extracting: reddit-images/g1190_cln9xzr.jpg  
 extracting: reddit-images/g1190_cln97xm.jpg  
 extracting: reddit-images/g1190_clna260.jpg  
 extracting: reddit-images/g1190_clna2x2.jpg  
 extracting: reddit-images/g1190_clna91w.jpg  
 extracting: reddit-images/g1190_clnad42.jpg  
 extracting: reddit-images/g1190_clnajd7.jpg  
 extracting: reddit-images/g1190_clnapoy.jpg  
 extracting: reddit-images/g1190_clnarjl.jpg  
 extracting: reddit-images/g1190_clnavnu.jpg  
 extracting: reddit-images/g1190_clnbalu.jpg  
 extracting: reddit-images/g1190_clnbf07.jpg  
 ext

### Load all images into evadb

In [ ]:
response = cursor.query("DROP TABLE IF EXISTS reddit_dataset;").df()
cursor.query(
    "LOAD IMAGE 'reddit-images/*.jpg' INTO reddit_dataset;"
).df()


,0
0,Number of loaded IMAGE: 34


### Register a SIFT FeatureExtractor 
It uses `kornia` library to extract sift features for each image

In [ ]:
cursor.query("DROP UDF IF EXISTS SiftFeatureExtractor;").df()
cursor.query("""CREATE UDF IF NOT EXISTS SiftFeatureExtractor
                    IMPL  '../evadb/udfs/sift_feature_extractor.py'""").df()

,0
0,UDF SiftFeatureExtractor successfully added to...


In [ ]:
# Keep track of which image gets the most votes
from collections import Counter
vote = Counter()

## Image-level similarity search pipeline. 
This pipeline creates one vector per image. Next, we should breakdown steps how we build the index and search similar vectors using the index.

In [ ]:
#1. Create index for the entire image
cursor.query("""DROP INDEX IF EXISTS reddit_sift_image_index""").df()
cursor.query("""CREATE INDEX reddit_sift_image_index 
                    ON reddit_dataset (SiftFeatureExtractor(data)) 
                    USING FAISS""").df()

06-24-2023 12:06:16 WARNING[drop_object_executor:drop_object_executor.py:_handle_drop_index:0106] Index reddit_sift_image_index does not exist, therefore cannot be dropped.


,0
0,Index reddit_sift_image_index successfully add...


In [ ]:
#2. Search similar vectors
response = cursor.query("""SELECT name FROM reddit_dataset ORDER BY
                    Similarity(
                      SiftFeatureExtractor(Open('reddit-images/g1190_clna260.jpg')),
                      SiftFeatureExtractor(data)
                    )
                    LIMIT 5""").df()

In [ ]:
#3. Update votes
for i in range(len(response)):
    vote[response["reddit_dataset.name"][i]] += 1
print(vote)

Counter({'reddit-images/g1190_clna260.jpg': 1, 'reddit-images/g1190_clndsnu.jpg': 1, 'reddit-images/g1190_clna91w.jpg': 1, 'reddit-images/g1190_clnc4uy.jpg': 1, 'reddit-images/g1190_cln97xm.jpg': 1})


## Object-level similarity search pipeline. 
This pipeline detects objects within images and generates vectors exclusively from the cropped objects. The index is then constructed using these vectors. To showcase the versatility of `evadb`, we leverage `Qdrant` vector store specifically for building this index. This demonstrates how seamlessly you can leverage different vector stores within evadb.

### 1. Extract all the object using `Yolo` from the images

In [ ]:
create_index_query = """
    CREATE MATERIALIZED VIEW IF NOT EXISTS 
      reddit_object_table (name, data, bboxes,labels)
      AS SELECT name, data, bboxes, labels FROM reddit_dataset
      JOIN LATERAL UNNEST(Yolo(data)) AS Obj(labels, bboxes, scores)"""
cursor.query(create_index_query).df()

06-24-2023 12:06:28 ERROR [base_model:base_model.py:_commit:0097] Database commit failed : (sqlite3.OperationalError) database is locked
(Background on this error at: https://sqlalche.me/e/14/e3q8)


06-24-2023 12:06:28 ERROR [base_model:base_model.py:save:0058] Database save failed : (sqlite3.OperationalError) database is locked
(Background on this error at: https://sqlalche.me/e/14/e3q8)


06-24-2023 12:06:28 ERROR [plan_executor:plan_executor.py:execute_plan:0172] Error while upserting entry to UdfCostCatalog: Error while inserting entry to UdfCostCatalog: (sqlite3.OperationalError) database is locked
(Background on this error at: https://sqlalche.me/e/14/e3q8)
Traceback (most recent call last):
  File "/home/jarulraj3/eva/test_evadb/lib/python3.10/site-packages/sqlalchemy/engine/base.py", line 1094, in _commit_impl
    self.engine.dialect.do_commit(self.connection)
  File "/home/jarulraj3/eva/test_evadb/lib/python3.10/site-packages/sqlalchemy/engine/default.py", line 686, in do_commit
    dbapi_connection.commit()
sqlite3.OperationalError: database is locked

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/jarulraj3/eva/test_evadb/lib/python3.10/site-packages/evadb/catalog/services/udf_cost_catalog_service.py", line 41, in insert_entry
    udf_obj.save(self.session)
  File "/home/jarulraj3/eva/test_

ExecutorError: Error while upserting entry to UdfCostCatalog: Error while inserting entry to UdfCostCatalog: (sqlite3.OperationalError) database is locked
(Background on this error at: https://sqlalche.me/e/14/e3q8)

### 2. Build an index on the feature vectors of the extracted objects

In [ ]:
cursor.query("""CREATE INDEX reddit_sift_object_index
                    ON reddit_object_table (SiftFeatureExtractor(Crop(data, bboxes)))
                    USING QDRANT""").df()

In [ ]:
# Create a cropped images (We are actively working on features to allow
# us to not do this outside SQL)
response = (
    cursor.query(
        "LOAD IMAGE 'reddit-images/g1190_clna260.jpg' INTO reddit_search_image_dataset"
    )
    .df()
)
print(response)
response = (
    cursor.query("SELECT Yolo(data).bboxes FROM reddit_search_image_dataset;")
    .df()
)
print(response)


import cv2
import pathlib

bboxes = response["yolo.bboxes"][0]

img = cv2.imread("reddit-images/g1190_clna260.jpg")
pathlib.Path("reddit-images/search-object/").mkdir(parents=True, exist_ok=True)
for i, bbox in enumerate(bboxes):
    xmin, ymin, xmax, ymax = bbox
    xmin, ymin, xmax, ymax = int(xmin), int(ymin), int(xmax), int(ymax)
    cropped_img = img[ymin:ymax, xmin:xmax]
    cv2.imwrite(f"reddit-images/search-object/search-{i}.jpg", cropped_img)


### 3. Retrieve using object-level similarity search

In [ ]:
#4. 
import os

for path in os.listdir("reddit-images/search-object/"):
    path = "reddit-images/search-object/" + path
    query = f"""SELECT name FROM reddit_object_table ORDER BY
                        Similarity(
                          SiftFeatureExtractor(Open('{path}')),
                          SiftFeatureExtractor(data)
                        )
                        LIMIT 1"""
    response = cursor.query(query).df()
    for i in range(len(response)):
        vote[response["reddit_object_table.name"][i]] += 0.5

    print(response)

## Combine the scores from image level and object level similarity to show similar images

In [ ]:
# !pip install matplotlib
import matplotlib.pyplot as plt

# Display top images
vote_list = list(reversed(sorted([(path, count) for path, count in vote.items()], key=lambda x: x[1])))
img_list = [path for path, _ in vote_list]

fig, ax = plt.subplots(nrows=1, ncols=6, figsize=[18,10])
ax[0].imshow(cv2.imread("reddit-images/g1190_clna260.jpg"))
ax[0].set_title("Search")

for i in range(5):
    axi = ax[i + 1]
    img = cv2.imread(img_list[i])
    axi.imshow(img)
    axi.set_title(f"Top-{i + 1}")

plt.show()